# Description

This Jupyter notebook example demonstrates how to run a model of the EMULF Delta Floater, exposed to time-dependent wave loads coming from Sima coupled analysis results via load reconstruction in Wasim, using OneWorkflow locally or in the cloud (**note**: cloud support will be added soon!)

Highlighted features of this example are:
- Read control data from Excel spreadsheet into a dictionary (Python), including file names for coupled analysis result, as well as start and stop times for Wasim load reconstruction and load transfer. More control data can easily be added to the spreadsheet and transferred to relevant input files using [`CreateInputFileFromFileTemplateCommand` (OneWorkflow)](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.oneworkflow.html#dnv.oneworkflow.create_input_file_from_file_template_command.CreateInputFileFromFileTemplateCommand). Some template control data are hardcoded by user in the notebook.
- Apply safety load factors. Load factors are first read from the spreadsheet, then modified and applied to the loads resulting from two Wasim runs, where one (total) loadcase includes all load effects and one (calm sea) load case includes only permanent loads. A single Sestra analysis is then run using the combined loads. The modified factors are applied to separate L#-files and then combined in sestra.inp using the MULL card. This is 
- One row set up for short time period load reconstruction and/or short time period load transfer only. Repeat rows for multiple events and/or seeds.
- Threshold value limiting output of usage factor results. See options in [`SesamCoreCommand`](https://myworkspace.dnv.com/download/public/sesam/workflow/docs/latest/source/dnv.sesam.commands.html#dnv.sesam.commands.sesam_core_command.SesamCoreCommand).
- Control Sesam Core run using commands in journal (JNL) file.
- Graphs of reaction force in z-direction from Sestra are presented for each load case.
- Buckling results are presented as a table of worst usage factor over all time steps and load cases for each stiffened plate field. Note that the table omits the two first time steps, to avoid unrealistic result peaks.
- Buckling results are also shown for a user selected number of graphs starting from a user selected usage factor as time-history plots of usage factor of all stiffeners in each stiffened plate field. 

Work in progress:
- To be added: Separate start and stop of buckling calculation in Sesam Core command journal file . Sestra is run for all time steps with load mapping.

The capacity model comes as a single JSON file from GeniE, and results are currently stored in one .csv and one .lis file for each stiffened plate field.


![EMULF Deltafloater](EMULF_Deltafloater.png)

# OneWorkflow setup

- Specify <font color='red'>root_folder / workspaceId</font>.
- Specify <font color='red'>oneWorkflowTMPFolder</font>.
- Use <font color='red'>cloudRun</font> variable to control running locally or in the cloud. (**note**: Let this be `False` until cloud support has been added and you have a valid cloud subscription.)
- Specify Excel file with load case names, start/stop/time step info and safety load factors for the input_data variable (<font color='red'>control_input2.xlsx</font>).

In [ ]:
from pathlib import Path
import os, sys
import pandas as pd
sys.path.append("../PythonModules")
root_folder = os.getcwd()

#User defined setup, see comments above
workspacePath = str(Path(root_folder, 'Workspace'))
workspaceId = "EMULF_DeltaFloater_Sesam_ULS"
cloudRun = False
input_data = pd.read_excel(os.path.join(workspacePath, "control_input2.xlsx"), index_col=1)


In [ ]:
from dnv.oneworkflow.utils import *
# local workspace, all results will be put here in the folder Loadcases, after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFiles
# If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath)


# Read Excel input, update template parameters in .inp files, run Wasim and Sesam Core

Files and file locations:
- Sima coupled analysis results files in the *LoadCases\\<load_case_name_from_excel>\\* folder. Results for a separate calm sea load case is required for the permanent load factor G.
- All other model and input, including template files are found in the *CommonFiles* folder

In [ ]:
#remove preivous existing resulte before running the analysis
import shutil
clean = True
if clean:
    shutil.rmtree(os.path.join(workspacePath,"LoadCases"), ignore_errors=True)
shutil.copytree(os.path.join(workspacePath,"Input"),os.path.join(workspacePath),dirs_exist_ok=True)

In [ ]:
#import pandas as pd
import os
from dnv.sesam.commands import *
from dnv.oneworkflow import *
import shutil
from WasimTaskCreator import *
import json

# Template parameters hardcoded by user
topsuper = 1 #top superelement number
prefix_stru = "STR_" #prefix for structure files
fixed_parameters_all_load_cases = {
    'mor_topsel': topsuper,
    'prefix': f"{prefix_stru}        ", #prefix, left-justified
    'prefix_stru': prefix_stru, #prefix, right-justified
    'prefix_struCalmSea': f'{prefix_stru}CalmSea',
    'prefix_struTotal' : f'{prefix_stru}Total'
} 


try:
    os.chdir(os.path.join(workspacePath,"LoadCases"))
except:
    print("LoadCases folder not found")
workflow_sequence = []
shutil.copytree(os.path.join(workspacePath,"Input"),os.path.join(workspacePath),dirs_exist_ok=True)

#Recognize the following column titles in Excel spreadsheet and map them to the correct input parameters given in Wasim template files
parameter_mapping = {'Group': "group",
				'LoadCase': "loadcase",
				"SimaFilesFolder": "sima_files_folder",
				"SimaFilesName": "sima_files_name",
                "TimeStep": "timestep",
				"Depth": "depth",
				"StartWasimSolve": "start_solve",
				"StopWasimSolve": "stop_solve",
            	"StartWasimStru": "start_stru",
				"StopWasimStru": "stop_stru",
				"NSteps": "nsteps",
				"StartSestra": "start_sestra",
				"StopSestra": "stop_sestra",
				"StartBuckling": "BUCKLINGSTART",
				"StopBuckling": "BUCKLINGEND",
				"LoadFactorSet": "loadfactorset",
                "fac_G": "fac_g",
                "fac_E": "fac_e",
                }

#Display data read from Excel
display(input_data)

#Loop over all load cases
for casename, case in input_data.iterrows():
    casedict = case.to_dict()
    input_parameters = {}
    for key, value in case.items():
        input_parameters[parameter_mapping[key]] = str(value)
    
    #Calculate modified load factors since Wasim doesn't distinguish static and dynamic loads
    input_parameters["fac_dyn"] = float(input_parameters['fac_e'])
    input_parameters["fac_calm"] = float(input_parameters['fac_g']) - float(input_parameters['fac_e'])

    input_parameters = input_parameters | fixed_parameters_all_load_cases

    print("The following parameters are used for load case: " + casename)
    print(json.dumps(input_parameters, indent=4, sort_keys=True))

    #Run Wasim including update of .inp files, for ULS including load factors
    tasks = WasimTaskCreator().CreateTasks(input_parameters)
    calm_sea_tasks =  WasimTaskCreator().CreateTasks(input_parameters, "CalmSea")
    tasks.extend(calm_sea_tasks)
    #Apply load factors
    fac_dyn = input_parameters["fac_dyn"]
    fac_calm=input_parameters["fac_calm"]
    # Update S#-files created by Wasim_stru, edit LCOM and append SCAL card
    scaling_command_dyn = PythonCommand(
            directory=workflow_client.common_directory,
            filename="appendScalingFactor.py",
            args=f"{prefix_stru}TotalS1.FEM {fac_dyn}"
            )
    tasks.append(scaling_command_dyn)

    scaling_command_calm = PythonCommand(
            directory=workflow_client.common_directory,
            filename="appendScalingFactor.py",
            args=f"{prefix_stru}CalmSeaS1.FEM {fac_calm}"
            )
    tasks.append(scaling_command_calm)
    #Update Sestra .inp file
    sestra_template_command = CreateInputFileFromFileTemplateCommand(
        template_input_file  = "sestra_template.inp",
        input_filename  = "sestra.inp",
        parameters= input_parameters
    )
    tasks.append(sestra_template_command)    
    
    #Update Sesam Core .jnl file
    sesam_core_template_command = CreateInputFileFromFileTemplateCommand(
        template_input_file  = "SesamCore_buckling_template.jnl",
        input_filename  = "SesamCore_buckling.jnl",
        parameters= input_parameters
    )
    tasks.append(sesam_core_template_command)
    
    #Run Sesam Core
    tasks.append(SesamCoreCommand(command = "ULS",input_file_name= "input.json", options = "-v"))
    
    #Run the above defined sequence of tasks
    workflow_sequence.append(CommandInfo(commands=tasks,load_case_foldername=casename))
    
    
print("Running commands in parallel")
await run_managed_commands_in_parallel_async(
             client=workflow_client,
             commands_info=workflow_sequence,
             log_job=False,
             files_to_download_from_blob_to_client=FileTransferOptions(max_size="11124MB",patterns=["**/*sestra.inp", "**/wasim_setup.inp", "**/wasim_solve.inp", 
                                                                                                    "**/wasim_snapshots.inp", "**/wasim_stru.inp", "**/*.csv", 
                                                                                                    "**/*.lis", "**/*.lis.bak", "**/*.mlg", "**/*.sin", "**/*.FEM", "**/*.jnl"]),
             enable_common_files_copy_to_load_cases=True,
 )


### Check if analysis is successful


In [ ]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")
print(current_time, current_date)

#Search different log file types for different messages
#Report only not successful log files

local__result_path = Path(workspacePath, workflow_client.results_directory)
os.chdir(local__result_path)
print("Verifying Wasim results:")
wasim_succeeded_text = "FINISHED: SUCCESS"
wasim_files_to_check = {"Wasim solve":'wasim_solve.lis.bak', "Wasim stru" : 'wasim_stru.lis.bak'}
import analysisStatusChecker
analysisStatusChecker.checkStatus(wasim_files_to_check,wasim_succeeded_text)
wasim_succeeded_text = "FINISHED: SUCCESS"
wasim_files_to_check = {"Wasim solve":'wasim_solve.lis', "Wasim stru" : 'wasim_stru.lis'}
analysisStatusChecker.checkStatus(wasim_files_to_check,wasim_succeeded_text)

print("Verifying Sesam Core results:")
score_succeeded_text = "Duration:"
score_to_check = {"Sesam Core":'SCORE.MLG'}
analysisStatusChecker.checkStatus(score_to_check,score_succeeded_text)


# Post-processing
Quality checks, maximum usage factors and criterion etc


In [ ]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")
print(current_time, current_date)

def format_time(value, _):
    return "{:.1e}".format(value)  # Format the time with two decimal places

#Loop over all load cases and plot graph of reaction force vs time for each load case
for loadcase_folder_name, _ in input_data.iterrows():
    args=f"{prefix_stru}_reactions_history1.csv" #from Sestra DREA card
    args = "STR__reactions_history1.csv"
    #currently hardcoded name, but should pick up prefix from previous definition
    result_path = os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name, args)
    print(result_path)
    reaction_history = pd.read_csv(result_path)
    df = pd.read_csv(result_path, delimiter=';') 
    from matplotlib import pylab as plt
    from matplotlib.ticker import FuncFormatter
    # Set the default font size for all labels

    time = df['Time']
    fx = df['FX']
    fy = df['FY']
    fz = df['FZ']
    mx = df['MX']
    my = df['MY']
    mz = df['MZ']

    # Define a custom formatting function for x-axis values
   

    # Plot a 2D graph
    plt.plot(time, fz, marker='.', label='FZ')

    # Add labels and title
    plt.xlabel('Result case id', fontsize=10)
    plt.ylabel('Force', fontsize=10)
    plt.xticks(fontsize=10)
    plt.gca().yaxis.set_major_formatter(FuncFormatter(format_time))
    plt.yticks(fontsize=10)
    plt.title('Reaction z-force over time-history', fontsize=14)
    plt.legend(fontsize=10)  # Show legend with labels

    # Add gridlines
    plt.grid(True, linestyle='--', alpha=0.7)

    # Set x-axis limits
    plt.xlim(min(time), max(time))


    # Show the plot
    plt.show()
    plt.savefig(os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name,"reaction_forces_plot.png"))

### Print maximum usage factor and criterion for worst stiffener on each platefield, over all load cases

In [ ]:
import os
from pathlib import Path
import glob
import pandas as pd

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")
print(current_time, current_date)

panels_and_cases = pd.DataFrame()
panel_names = set()
for loadcase_folder_name, _ in input_data.iterrows():
    result_folder = os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name)
        
    os.chdir(result_folder)

    # loop over all buckling result files (csv)
    
    schema={
        "math score": int
    }
    
    #.csv file runname is based on name given in SesamCore_buckling_template.jnl on RUN BUCKLING-CHECK line
    for file in glob.glob('SesamCore_runname1_*Panel*.csv'):
        gen = pd.read_csv(file, dtype=schema, chunksize=10000000)
        # Set threshold value for removing the first few result cases to omit initial extreme results
        panel = pd.concat((x.query("`Result case id` >= 3") for x in gen), ignore_index=True)
        panel['Plate field'] = Path(file).stem
        panel['LoadCase'] = loadcase_folder_name
        if not panel.empty:
            panels_and_cases = pd.concat([panels_and_cases,panel])
            panel_names.add(Path(file).stem)

#Show table of worst usage factors
df2 = pd.DataFrame(columns=['Stiffener name','Plate field', 'UfMax','UfMax criterion','LoadCase', 'Time'])
for panel_name in panel_names:
    df = panels_and_cases.query('`Plate field` == @panel_name')
    if not df.empty:
        df2.loc[len(df2.index)] = dict(df.iloc[df['UfMax'].idxmax()])
df2.sort_values(by='UfMax',ascending=False,inplace=True)
print(f"Maximum Uf for each panel, taken over all load cases and result cases.")
print(f"Note that the first two time steps (usually extreme results) have been filtered out to avoid reporting unrealistic peak results.")
display(df2)


### Plot usage factors for each plate field (all stiffeners) over time-history of worst load case 

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display
import numpy as np
import pandas as pd
import glob
from ipywidgets import interactive, Text, Dropdown, Layout
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
current_date = now.strftime("%d/%m/%Y")
print(current_time, current_date)

# Number of graphs to display
number_of_plate_fields_to_display =10
# Result threshold value, e g show the first 10 graphs with maximum usage factors higher than threshold value
threshold_for_uf = 1.0 
filtered_df = df2[df2['UfMax'] < threshold_for_uf]
highet_usage_factor =filtered_df.head(number_of_plate_fields_to_display)
plate_fields_with_highest_uf= highet_usage_factor.head(number_of_plate_fields_to_display)["Plate field"].to_list()
display(highet_usage_factor)
def multiplot(PlateField, LoadCase):
    import matplotlib as mpl

    # Set the default font size for all labels
    mpl.rcParams.update({'font.size': 16})
    stiffenernames = data_frames[PlateField]['Stiffener name'].unique()
    category_colors = dict(zip(stiffenernames, cm.rainbow(np.linspace(0, 1, len(stiffenernames)))))

    ax= data_frames[PlateField].plot(kind='scatter', x='Time', y='UfMax',  c=data_frames[PlateField]['Stiffener name'].apply(lambda x: category_colors[x]), figsize=(20, 7))
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=category)
           for category, color in category_colors.items()]
    ax.legend(handles=handles, title='Stiffener name')
    ax.set_title('UfMax over time-history for worst load case. Inlcudes all time steps, also those omitted in max table reporting.')
    fig = ax.get_figure()
    fig.savefig("UfMaxplot.png")

# for loadcase_folder_name, _ in df_cases.iterrows():
for loadcase_folder_name in highet_usage_factor['LoadCase'].unique():
    result_folder = os.path.join(workspacePath,workflow_client.results_directory, loadcase_folder_name)
        
    os.chdir(result_folder)
    data_frames = {}
    for file in glob.glob('SesamCore_runname1_*Panel*.csv'):
        try:
            data_frames[Path(file).stem] = pd.read_csv(file)[['Time', 'UfMax', 'Stiffener name']]
        except:
            print(file + " had issues with formatting\n")
    
    if not data_frames:
        print("No valid cases found")
    else:
        interactive_plot = interactive(multiplot, PlateField=data_frames.keys(), LoadCase=loadcase_folder_name)
        display(interactive_plot)